# 01 - Train Baseline MobileNetV2 on CIFAR-10

This notebook trains a baseline MobileNetV2 model on the CIFAR-10 dataset.  
It evaluates the model's accuracy and saves the trained model for further optimization.


In [20]:
# 2 Import libraries

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import os
from time import time


In [21]:
# 3 Setup drvice
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")




Using device: cpu


In [22]:
# 4 Load and Transform CIFAR-10

import ssl
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data

# Define transformations
transform = transforms.Compose([
    transforms.Resize(224),  # Required input size for MobileNetV2
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset from extracted files
trainset = datasets.CIFAR10(root='../data', train=True, download=False, transform=transform)
trainloader = data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = datasets.CIFAR10(root='../data', train=False, download=False, transform=transform)
testloader = data.DataLoader(testset, batch_size=32, shuffle=False)

print("✅ CIFAR-10 loaded successfully!")


✅ CIFAR-10 loaded successfully!


In [23]:
#5 Load and Modify MobileNetV2
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 10)  # CIFAR-10 = 10 classes
model = model.to(device)


In [24]:
# 6 Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [25]:
#7 Training Loop
# Shortened Training Loop (for limited CPU devices)
EPOCHS = 2
max_batches = 20  # Limit training to 20 batches only

for epoch in range(EPOCHS):
    running_loss = 0.0
    model.train()
    
    for i, (images, labels) in enumerate(trainloader):
        if i >= max_batches:
            break

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"✅ Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss:.4f}")



✅ Epoch 1/2, Loss: 31.2258
✅ Epoch 2/2, Loss: 26.3782


In [26]:
#8: Evaluate Accuracy
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 58.01%


In [27]:
#9: Save the Trained Model
os.makedirs('../models', exist_ok=True)
torch.save(model.state_dict(), '../models/mobilenetv2_cifar10_baseline.pth')
print("Model saved to '../models/mobilenetv2_cifar10_baseline.pth'")


Model saved to '../models/mobilenetv2_cifar10_baseline.pth'
